In [2]:
from cassandra.cluster import Cluster

In [64]:
cluster = Cluster(['localhost'], port=9042) # You can specify multiple contact points if needed
session = cluster.connect() 

In [19]:
session.execute('''CREATE KEYSPACE IF NOT EXISTS cbd WITH replication 
                = {'class': 'SimpleStrategy', 'replication_factor': 1}''')


In [115]:
session.execute("DROP TABLE IF EXISTS cbd.inscritos")

In [116]:
session.execute('''
    CREATE TABLE cbd.inscritos (
    natureza_do_estabelecimento VARCHAR,
    nome_estabelecimento VARCHAR,
    tipo_de_ensino VARCHAR,
    nivel_de_formacao VARCHAR,
    curso VARCHAR,
    distrito VARCHAR,
    sexo VARCHAR,
    total INT,
    grande_grupo VARCHAR,
    pais VARCHAR,
    ano INT,
    PRIMARY KEY (nome_estabelecimento,nivel_de_formacao,curso,distrito,sexo,ano)
);

''')

In [78]:
session.execute('TRUNCATE cbd.inscritos')

In [117]:
import glob
import pandas as pd
import os
import re
import uuid

pd.options.mode.chained_assignment = None

dict_colunas = {'Subsistema de Ensino':'natureza_do_estabelecimento',
                'Estabelecimento':'nome_estabelecimento',
                'Tipo de Ensino':'tipo_de_ensino',
                'Nível de Formação':'nivel_de_formacao',
                'Curso':'curso',
                'Distrito':'distrito',
                'Sexo':'sexo',
                'Total':'total',
                'CNAEF - Grande Grupo':'grande_grupo',
                'País de Nacionalidade':'pais',
                'N.º de alunos inscritos':'total',
                'N':'total'}

path = r'C:\Users\bruno\Desktop\Projeto cbd\Inscritos'

excel_files = glob.glob(path + '/*.xlsx')

pattern = r'\d+'


for idx, file in enumerate(excel_files, 1):
    # Create an ExcelFile object without reading the file into memory
    print('Dataset:',idx)
    df = pd.read_excel(file)
    df.rename(columns=dict_colunas, inplace=True)
    df_filtered = df[[col for col in df.columns if col in dict_colunas.values()]]
    match = re.search(pattern, file)
    ano = int(match.group())
    df_filtered['ano'] = ano
    df_filtered = df_filtered[:-2]
    df_filtered.replace({pd.NA: None}, inplace=True)
    columns = list(df_filtered.columns)
    columns_insert=','.join(columns) 
    df_filtered['total'] = df_filtered['total'].astype(int)
    for _, row in df_filtered.iterrows():
        # Escape special characters and enclose string values in single quotes
    
        row_values = [f"'{val}'" if isinstance(val, str) else str(val) for val in row.values]
        
        # Construct the INSERT query dynamically for each row
        query = f'''INSERT INTO cbd.inscritos ({columns_insert}) VALUES ({','.join(row_values)});'''
        
        session.execute(query)

    
    


Dataset: 1


KeyboardInterrupt: 

In [126]:
# Execute the query
rows = session.execute('''
SELECT ano, distrito, SUM(total) AS n
FROM cbd.inscritos
GROUP BY distrito
ORDER BY distrito;
''')

# Convert the rows to a list and sort them based on total_enrollment
sorted_rows = sorted(rows, key=lambda x: x.n, reverse=True)

# Print the sorted rows
for row in sorted_rows:
    print(row.distrito, row.n)


InvalidRequest: Error from server: code=2200 [Invalid query] message="Group by currently only support groups of columns following their declared order in the PRIMARY KEY"